In [1]:
import random
import tensorflow as tf
import numpy as np
import string
import re
from sklearn.model_selection import train_test_split

In [2]:
file_loc="/content/spa.txt"
with open(file_loc) as f:
    lines=f.read().split("\n")[:-1]
data_pairs=[]
for line in lines:
    english, spanish=line.split("\t")
    spanish="[start] "+spanish+" [end]"
    data_pairs.append((english,spanish))

In [3]:
len(data_pairs)

118964

In [4]:
random.shuffle(data_pairs)

In [5]:
data_pairs

[('It was a terrible accident.', '[start] Fue un accidente atroz. [end]'),
 ("I think it's a possibility.", '[start] Es una posibilidad. [end]'),
 ('A few months later, Israel invaded Egypt.',
  '[start] Pocos meses más tarde, Israel invadió Egipto. [end]'),
 ('Tom is working on another novel.',
  '[start] Tom está trabajando en otra novela. [end]'),
 ('The dawn is breaking.', '[start] Está amaneciendo. [end]'),
 ('Milk boils at a higher temperature than water.',
  '[start] La leche hierve a más alta temperatura que el agua. [end]'),
 ('Watch yourselves.', '[start] Tengan cuidado. [end]'),
 ("It looks like he'll be coming here next month.",
  '[start] Parece que él vendrá acá el próximo mes. [end]'),
 ('It hardly ever rains in the area.',
  '[start] Casi nunca llueve en esta región. [end]'),
 ('This amount includes tax.', '[start] Este monto incluye impuestos. [end]'),
 ('Tom ran down the stairs.',
  '[start] Tom bajó corriendo las escaleras. [end]'),
 ('I have no money to pay for the 

In [6]:
train_data,test_data=train_test_split(data_pairs)

In [7]:
train_data, val_data=train_test_split(train_data)

In [8]:
print(len(train_data),len(test_data),len(val_data))

66917 29741 22306


In [9]:
strip_chars=string.punctuation+"¿"
strip_chars=strip_chars.replace("[","")
strip_chars=strip_chars.replace("]","")

In [10]:
strip_chars

'!"#$%&\'()*+,-./:;<=>?@\\^_`{|}~¿'

In [11]:
f"[{re.escape(strip_chars)}]"

'[!"\\#\\$%\\&\'\\(\\)\\*\\+,\\-\\./:;<=>\\?@\\\\\\^_`\\{\\|\\}\\~¿]'

In [12]:
from tensorflow.keras import layers
def customStandardisation(input_string):
    lowercase=tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase,f"[{re.escape(strip_chars)}]","")

In [13]:
vocab_size=15000
sequence_length=20

In [14]:
english_vectorization=layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

In [15]:
spanish_vectorization=layers.experimental.preprocessing.TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length+1,
    standardize=customStandardisation)

In [16]:
train_english_texts=[pair[0] for pair in train_data]
train_spanish_texts=[pair[1] for pair in train_data]

In [17]:
english_vectorization.adapt(train_english_texts)
spanish_vectorization.adapt(train_spanish_texts)

In [18]:
english_vectorization

In [19]:
batch_size=64
def format_dataset(eng,spa):
    eng=english_vectorization(eng)
    spa=spanish_vectorization(spa)
    return ({"english":eng,"spanish":spa[:,:-1],},spa[:,1:])
def make_dataset(pairs):
    eng_texts,spa_texts=zip(*pairs)
    eng_texts=list(eng_texts)
    spa_texts=list(spa_texts)
    dataset=tf.data.Dataset.from_tensor_slices((eng_texts,spa_texts))
    dataset=dataset.batch(batch_size)
    dataset=dataset.map(format_dataset,num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

In [20]:
train_ds=make_dataset(train_data)
val_ds=make_dataset(val_data)

In [21]:
print(list(train_ds.as_numpy_iterator())[50])

({'english': array([[   5,  101, 1630, ...,    0,    0,    0],
       [2452,   23, 1172, ...,    0,    0,    0],
       [  40,    3,  145, ...,    0,    0,    0],
       ...,
       [  21,  251,   37, ...,    0,    0,    0],
       [  21,   14, 5315, ...,    0,    0,    0],
       [   3,  496,   10, ...,    0,    0,    0]]), 'spanish': array([[   2,   50,   12, ...,    0,    0,    0],
       [   2,   20, 3088, ...,    0,    0,    0],
       [   2,   54,  172, ...,    0,    0,    0],
       ...,
       [   2,   26,  136, ...,    0,    0,    0],
       [   2,   26,   15, ...,    0,    0,    0],
       [   2,   35, 1094, ...,    0,    0,    0]])}, array([[   50,    12,   529, ...,     0,     0,     0],
       [   20,  3088,    15, ...,     0,     0,     0],
       [   54,   172,    56, ...,     0,     0,     0],
       ...,
       [   26,   136,    18, ...,     0,     0,     0],
       [   26,    15, 11189, ...,     0,     0,     0],
       [   35,  1094,    11, ...,     0,     0,     0]]

In [22]:
embed_dim = 256
latent_dim = 1024

In [23]:
spa_vocab = spanish_vectorization.get_vocabulary()

In [24]:
spa_vocab

['',
 '[UNK]',
 '[start]',
 '[end]',
 'de',
 'que',
 'a',
 'no',
 'tom',
 'la',
 'el',
 'en',
 'es',
 'un',
 'me',
 'se',
 'por',
 'lo',
 'una',
 'Él',
 'los',
 'su',
 'está',
 'mi',
 'con',
 'le',
 'ella',
 'qué',
 'te',
 'para',
 'mary',
 'y',
 'las',
 'más',
 'al',
 'yo',
 'tu',
 'estoy',
 'muy',
 'eso',
 'este',
 'tiene',
 'esta',
 'del',
 'quiero',
 'él',
 'estaba',
 'aquí',
 'tengo',
 'fue',
 'si',
 'casa',
 'como',
 'hacer',
 'puedo',
 'todo',
 'algo',
 'ha',
 'esto',
 'hay',
 'tiempo',
 'gusta',
 'son',
 'todos',
 'favor',
 'ir',
 'tan',
 'puede',
 'vez',
 'nada',
 'mucho',
 'era',
 'cuando',
 'he',
 'bien',
 'ellos',
 'nos',
 'solo',
 'ser',
 'nunca',
 'trabajo',
 'estás',
 'sé',
 'mañana',
 'dónde',
 'sus',
 'ahora',
 'creo',
 'dinero',
 'dos',
 'pero',
 'cómo',
 'ya',
 'tienes',
 'dijo',
 'quién',
 'tomás',
 'están',
 'ese',
 'hace',
 'tú',
 'soy',
 'día',
 'siempre',
 'libro',
 'hablar',
 'puedes',
 'hoy',
 'había',
 'poco',
 'esa',
 'quiere',
 'va',
 'voy',
 'verdad',
 've

In [25]:
spa_index_lookup=dict(zip(range(len(spa_vocab)),spa_vocab))

In [26]:
max_decoded_sentence_length=20

In [27]:
class TransformerEncoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention=layers.MultiHeadAttention(
        num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=tf.keras.Sequential(
        [layers.Dense(dense_dim, activation='relu'),layers.Dense(embed_dim),])
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()
        
    def call(self,inputs,mask=None):
        if mask is not None:
            mask=mask[:,tf.newaxis,:]
        attention_output=self.attention(inputs,inputs,attention_mask=mask)
        proj_input=self.layernorm_1(inputs+attention_output)
        proj_output=self.dense_proj(proj_input)
        return self.layernorm_2(proj_input+proj_output)
#     def get_config(self)


In [28]:
class TransformerDecoder(layers.Layer):
    def __init__(self,embed_dim,dense_dim,num_heads,**kwargs):
        super().__init__(**kwargs)
        self.embed_dim=embed_dim
        self.dense_dim=dense_dim
        self.num_heads=num_heads
        self.attention_1=layers.MultiHeadAttention(
        num_heads=num_heads,key_dim=embed_dim)
        self.attention_2=layers.MultiHeadAttention(
        num_heads=num_heads,key_dim=embed_dim)
        self.dense_proj=tf.keras.Sequential(
        [layers.Dense(dense_dim,activation='relu'),
         layers.Dense(embed_dim),])
        self.layernorm_1=layers.LayerNormalization()
        self.layernorm_2=layers.LayerNormalization()
        self.layernorm_3=layers.LayerNormalization()
        self.supports_masking=True
        
#     def get_config(self):

    def get_casual_attention_mask(self,inputs):
        input_shape=tf.shape(inputs)
        batch_size, sequence_length=input_shape[0],input_shape[1]
        i=tf.range(sequence_length)[:,tf.newaxis]
        j=tf.range(sequence_length)
        mask=tf.cast(i>=j,dtype='int32')
        mask=tf.reshape(mask,(1,input_shape[1],input_shape[1]))
        mult=tf.concat(
        [tf.expand_dims(batch_size,-1),tf.constant([1,1],dtype=tf.int32)],axis=0)
        return tf.tile(mask,mult)
    
    def call(self,inputs,encoder_outputs,mask=None):
        casual_mask=self.get_casual_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:,tf.newaxis,:],dtype="int32")
            padding_mask=tf.minimum(padding_mask,casual_mask)
            
        attention_output_1=self.attention_1(
        query=inputs,
        value=inputs,
        key=inputs,
        attention_mask=casual_mask)
        
        attention_output_1=self.layernorm_1(inputs+attention_output_1)
        
        attention_output_2=self.attention_2(
        query=attention_output_1,
        value=encoder_outputs,
        key=encoder_outputs,
        attention_mask=padding_mask,)
        
        attention_output_2=self.layernorm_2(attention_output_1+attention_output_2)
        proj_output=self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2+proj_output)

In [29]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [30]:
embed_dim=256
dense_dim=2048
num_heads=8

encoder_inputs=tf.keras.Input(shape=(None,),dtype="int64",name='english')
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(encoder_inputs)
encoder_outputs=TransformerEncoder(embed_dim,dense_dim,num_heads)(x)

decoder_inputs = tf.keras.Input(shape=(None,),dtype="int64",name="spanish")
x=PositionalEmbedding(sequence_length,vocab_size,embed_dim)(decoder_inputs)
x=TransformerDecoder(embed_dim,dense_dim,num_heads)(x,encoder_outputs)

x=layers.Dropout(0.5)(x)

decoder_outputs=layers.Dense(vocab_size,activation="softmax")(x)
transformer=tf.keras.Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [31]:
transformer.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 english (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 spanish (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 positional_embedding (Position  (None, None, 256)   3845120     ['english[0][0]']                
 alEmbedding)                                                                                     
                                                                                                  
 positional_embedding_1 (Positi  (None, None, 256)   3845120     ['spanish[0][0]']            

In [33]:
transformer.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [34]:
stats=transformer.fit(train_ds,epochs=100,batch_size=32,validation_data=val_ds)

Epoch 1/100
1046/1046 [==============================] - 105s 88ms/step - loss: 3.6342 - accuracy: 0.4575 - val_loss: 2.4864 - val_accuracy: 0.5723
Epoch 2/100
1046/1046 [==============================] - 75s 72ms/step - loss: 2.3660 - accuracy: 0.5995 - val_loss: 1.9835 - val_accuracy: 0.6387
Epoch 3/100
1046/1046 [==============================] - 75s 72ms/step - loss: 1.8646 - accuracy: 0.6579 - val_loss: 1.8114 - val_accuracy: 0.6615
Epoch 4/100
1046/1046 [==============================] - 75s 72ms/step - loss: 1.5619 - accuracy: 0.6943 - val_loss: 1.7474 - val_accuracy: 0.6732
Epoch 5/100
1046/1046 [==============================] - 75s 72ms/step - loss: 1.3542 - accuracy: 0.7198 - val_loss: 1.7021 - val_accuracy: 0.6807
Epoch 6/100
1046/1046 [==============================] - 75s 71ms/step - loss: 1.2005 - accuracy: 0.7421 - val_loss: 1.7012 - val_accuracy: 0.6866
Epoch 7/100
1046/1046 [==============================] - 84s 80ms/step - loss: 1.0782 - accuracy: 0.7594 - val_loss: 

In [38]:
transformer.save("/content/model.h5")

In [ ]:
spa_vocab=spanish_vectorization.get_vocabulary()
spa_index_lookup=dict(zip(range(len(spa_vocab)),spa_vocab))
max_decoded_sentence_length=20

def decode_sequence(input_sentence):
    tokenized_input_sentence=spanish_vectorization([input_sentence])
    decoded_sentence="[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence=spanish_vectorization([decoded_sentence])[:,:,-1]
        predictions=transformer([tokenized_input_sentence,tokenized_target_sentence])
        sampled_token_index=np.argmax(predictions[0,i,:])
        sampled_token=spa_index_lookup(sampled_token_index)
        decoded_sentence+=" "+sampled_token
        if sampled_token=="[end]":
            break
        return decoded_sentence

test_eng_texts=[pair[0] for pair in test_data]

for _ in range(20):
    input_sentence=random.choice(test_eng_texts)